# TRAPpy: Interactive Plotting

<span style="font-size: 150%">Re Run the cells to generate the graphs</span>

In [1]:
import sys,os
sys.path.append("..")
import numpy.random
import pandas as pd
import shutil
import tempfile
import trappy
trace_thermal = "./trace.txt"
trace_sched = "../tests/raw_trace.dat"
TEMP_BASE = "/tmp"

In [2]:
def setup_thermal():
    tDir = tempfile.mkdtemp(dir="/tmp", prefix="trappy_doc", suffix = ".tempDir")
    shutil.copyfile(trace_thermal, os.path.join(tDir, "trace.txt"))
    return tDir

def setup_sched():
    tDir = tempfile.mkdtemp(dir="/tmp", prefix="trappy_doc", suffix = ".tempDir")
    shutil.copyfile(trace_sched, os.path.join(tDir, "trace.dat"))
    return tDir

In [3]:
temp_thermal_location = setup_thermal()
trace1 = trappy.FTrace(temp_thermal_location)

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:789: UserWarning: Reading from .txt file, .dat is preferred. Not only do .txt files occupy more disk space, it is also not possible to determine the format of the traces contained within them.
  warnings.warn(warnstr)


# Interactive Line Plotting of Data Frames

Interactive Line Plots Supports the same API as the LinePlot but provide an interactive plot that can be zoomed by clicking and dragging on the desired area. Double clicking resets the zoom.

We can create an interactive plot easily from a dataframe by passing the data frame and the columns we want to plot as parameters:

In [4]:
columns = ["tick", "tock"]
df = pd.DataFrame(numpy.random.randn(1000, 2), columns=columns).cumsum()

trappy.ILinePlot(df, column=columns).view()

""
""


## Plotting independent series

It is also possible to plot traces with different index values (i.e. x-axis values)

In [5]:
columns = ["tick", "tock", "bang"]
df_len = 1000
df1 = pd.DataFrame(numpy.random.randn(df_len, 3), columns=columns, index=range(df_len)).cumsum()
df2 = pd.DataFrame(numpy.random.randn(df_len, 3), columns=columns, index=(numpy.arange(0.5, df_len, 1))).cumsum()

In [6]:
trappy.ILinePlot([df1, df2], column="tick").view()

""
""


This does not affect filtering or pivoting in any way

In [7]:
df1["bang"] = df1["bang"].apply(lambda x: numpy.random.randint(0, 4))
df2["bang"] = df2["bang"].apply(lambda x: numpy.random.randint(0, 4))

In [8]:
trappy.ILinePlot([df1, df2], column="tick", filters = {'bang' : [2]}, title="tick column values for which bang is 2").view()

""
""


In [9]:
trappy.ILinePlot([df1, df2], column="tick", pivot="bang", title="tick column pivoted on bang column").view()

,
,
,
,


# Interactive Line Plotting of Traces

We can also create them from trace objects

In [10]:
map_label = {
        "00000000,00000006" : "A57",
        "00000000,00000039" : "A53",
}

l = trappy.ILinePlot(
        trace1,                                          # TRAPpy FTrace Object
        trappy.cpu_power.CpuInPower,                     # TRAPpy Event (maps to a unique word in the Trace)
        column=[                                         # Column(s)
            "dynamic_power",
            "load1"],

        filters={                                        # Filter the data  
            "cdev_state": [
                1,
                0]},
        pivot="cpus",                                    # One plot for each pivot will be created
        map_label=map_label,                             # Optionally, provide an alternative label for pivots
        per_line=1)                                      # Number of graphs per line
l.view()

""
""
""
""


You can also change the drawstyle to "steps-post" for step plots. These are suited if the data is discrete 
and linear interploation is not required between two data points

In [11]:
l = trappy.ILinePlot(
    trace1,                                          # TRAPpy FTrace Object
    trappy.cpu_power.CpuInPower,                     # TRAPpy Event (maps to a unique word in the Trace)
    column=[                                         # Column(s)
        "dynamic_power",
        "load1"],

    filters={                                        # Filter the data  
        "cdev_state": [
            1,
            0]},
    pivot="cpus",                                    # One plot for each pivot will be created

    per_line=1,                                      # Number of graphs per line
    drawstyle="steps-post")                                     
l.view()

""
""
""
""


## Plots for a given time range

Performance can suffer if ILinePlot tries to make a huge plot.  One way of fixing it is by limiting the period of time plotted using the `xlim` parameter:

In [12]:
trappy.ILinePlot(
    trace1,
    signals=["thermal:temp"],
    xlim=(1, 4),           # Only between seconds 1 and 4
).view()

""
""


## Synchronized zoom in multiple plots

ILinePlots can zoom all at the same time.  You can do so using the `group` and `sync_zoom` parameter.  All ILinePlots using the same group name zoom at the same time.  Note the use of signals with colors.

In [13]:
trappy.ILinePlot(
    trace1,
    signals=["cpu_in_power:dynamic_power:18,140,171", "cpu_in_power:load1:0xcf,0x36,0x4a"],
    pivot="cpus",
    group="synchronized",
    sync_zoom=True
).view()

,
,


## Styling ILinePlots

Use `fill=True` to colour-fill the space under the line. `fill_alpha` optionally sets the opacity.

In [14]:
trappy.ILinePlot(
    trace1,
    signals=["thermal:temp"],
    fill=True,
    fill_alpha=0.5
).view()

""
""


# EventPlot

TRAPpy's  Interactive Plotter features an Interactive Event TimeLine Plot. It accepts an input data of the type
<pre>
<code>
                   { "A" : [
                                    [event_start, event_end, lane],
                                    .
                                    .
                                    [event_start, event_end, lane],
                                 ],
                     .
                     .
                     .

                     "B" : [
                                    [event_start, event_end, lane],
                                    .
                                    .
                                    [event_start, event_end, lane],
                    .
                    .
                    .
                   }
                                
</code>
</pre>

Hovering on the rectangles gives the name of the process element and scrolling on the Plot Area and the window in the summary controls the zoom. One can also click and drag for panning a zoomed graph.

For Example:

In [15]:
A = [
    
        [0, 3, 0],
        [4, 5, 2],
]

B = [
        [0, 2, 1],
        [2, 3, 3],
        [3, 4, 0],
]

C =  [
        [0, 2, 3],
        [2, 3, 2],
        [3, 4, 1],
]

EVENTS = {}
EVENTS["A"] = A
EVENTS["B"] = B
EVENTS["C"] = C

trappy.EventPlot(EVENTS,
                 keys=EVENTS.keys,                     # Name of the Process Element
                 lane_prefix="LANE: ",                 # Name of Each TimeLine
                 num_lanes=4,                          # Number of Timelines
                 domain=[0,5]                          # Time Domain
                ).view()

Lane names can also be specified as strings (or hashable objects that have an str representation) as follows

In [16]:
A = [
    
        [0, 3, "zero"],
        [4, 5, "two"],
]

B = [
        [0, 2, 1],
        [2, 3, "three"],
        [3, 4, "zero"],
]

C =  [
        [0, 2, "three"],
        [2, 3, "two"],
        [3, 4, 1],
]

EVENTS = {}
EVENTS["A"] = A
EVENTS["B"] = B
EVENTS["C"] = C

trappy.EventPlot(EVENTS,
                 keys=EVENTS.keys,                     # Name of the Process Element
                 lanes=["zero", 1, "two", "three"],
                 domain=[0,5]                          # Time Domain
                ).view()

It is also possible to define a colour map to associate a specific colour to each event. A colour string can be:

 - a colour name, `green`, `red`, `blue`, etc.
 
 - the HEX representation of the colour, `#0000FF` for blue, `#FF0000` for red
 

In [17]:
# Using colour names
trappy.EventPlot(EVENTS,
                 keys=EVENTS.keys,                     # Name of the Process Element
                 lanes=["zero", 1, "two", "three"],
                 domain=[0,5],                         # Time Domain
                 color_map={"A" : "blue", "B" : "red", "C" : "green"}
                ).view()

In [18]:
# Using HEX representation of colours
trappy.EventPlot(EVENTS,
                 keys=EVENTS.keys,                     # Name of the Process Element
                 lanes=["zero", 1, "two", "three"],
                 domain=[0,5],                         # Time Domain
                 color_map={"A" : "	#ffa07a", "B" : "#f08080", "C" : "#add8e6"}
                ).view()

# TracePlot

A specification of the EventPlot creates a kernelshark like plot if the sched_switch event is enabled in the traces

In [19]:
f = setup_sched()
trappy.plotter.plot_trace(f)

IOError: [Errno 2] No such file or directory: '../tests/raw_trace.dat'

# Exporting notebooks with interactive plots to HTML

Notebooks with ILinePlot or EventPlot can't be exported to HTML using File->Download as->HTML.  They need to be converted from the command line:
```
jupyter nbconvert --to=trappy.nbexport.HTML notebook.ipynb
```
You need nbconvert >= 4.2 and trappy has to be in your `PYTHONPATH`.